
# Concept Drift Benchmark — Enhanced Baseline (v3)

**🎯 Mục đích**: Comprehensive benchmark của drift detection methods với evaluation framework nâng cao

**🔧 Detectors**: 
- **Custom**: ShapeDD (kernel-based MMD)
- **Traditional**: DDM, Page-Hinkley, ADWIN, MDDM, FHDDM/FHDDMS  
- **River Library**: EDDM, HDDM_A, HDDM_W, KSWIN

**📊 Datasets**: 
- **Synthetic**: SEA, Rotating Hyperplane, LED (abrupt/gradual), Interchanging RBF
- **Real-world**: Elec2, RandomRBFDrift

**📈 Advanced Metrics**: 
- **Detection Quality**: β-score, F1@AR, Accuracy@AR, Global Scores
- **Statistical Analysis**: Delay distributions, permutation tests
- **Visualization**: Timeline plots, comparative analysis

> **v3 Updates**: Tích hợp River library, metrics nâng cao, automated reporting, và framework tối ưu hóa



## 📊 Evaluation Framework & Metrics

### **🎯 Classification Performance (Prediction Quality)**
- **Prequential Accuracy**: \( \text{Acc} = \frac{\sum_{t} \mathbb{1}(\hat{y}_t = y_t)}{T} \) (predict-then-update protocol)
- **Macro-F1**: Balanced across classes to handle imbalanced streams
  - \( \text{Precision}_k = \frac{TP_k}{TP_k + FP_k} \), \( \text{Recall}_k = \frac{TP_k}{TP_k + FN_k} \)
  - \( F1_k = \frac{2\cdot \text{Prec}_k \cdot \text{Rec}_k}{\text{Prec}_k + \text{Rec}_k} \), **Macro-F1** = \( \frac{1}{K}\sum_k F1_k \)

### **🚨 Drift Detection Quality**
- **True Positives (TP)**: Detections matching ground truth drifts (within tolerance)
- **False Alarms (FA)**: Detections not matching any true drift
- **Detection Delay**: Mean latency from true drift to first alarm
- **β-score**: \( \frac{TP}{P + \beta \cdot FP} \) (β=0.5 balances precision/recall)

### **⚡ Advanced Metrics (v3 New)**
- **Alarm Rate (AR)**: \( \frac{\text{#Alarms}}{N} \times 10^4 \) per 10k samples
- **F1@AR**: \( F1 - \lambda \cdot AR \) (λ=0.01, penalizes excessive alarms)
- **Accuracy@AR**: \( Acc - \lambda \cdot AR \) (λ=0.01)
- **Global Scores**: Min-max normalized across datasets → macro-average

### **📈 Statistical Analysis**
- **Delay Distributions**: mean, median, percentiles
- **Permutation Tests**: p-value validation for statistical significance
- **Confidence Intervals**: Bootstrap analysis for robust results

> **🎯 Ideal Detector**: High Acc/F1, Low Delay/FA, Balanced F1@AR score


In [2]:

# 📦 Dependencies & Setup
import math, random, time, warnings
from dataclasses import dataclass
from typing import List, Tuple, Optional, Dict, Any
from pathlib import Path

# Core libraries
import numpy as np
import pandas as pd
from collections import deque, defaultdict

# Machine Learning
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# River library for drift detection (install if needed)
try:
    from river import drift as river_drift
    from river import tree, metrics as river_metrics
    RIVER_AVAILABLE = True
    print("✅ River library loaded successfully")
except ImportError:
    RIVER_AVAILABLE = False
    print("⚠️  River library not available. Install with: pip install river")

# Reproducibility
np.random.seed(42)
random.seed(42)
warnings.filterwarnings('ignore')

print("🚀 Enhanced Concept Drift Benchmark v3 - Setup Complete")


⚠️  River library not available. Install with: pip install river
🚀 Enhanced Concept Drift Benchmark v3 - Setup Complete


## 🌊 1) Enhanced Stream Generators Framework

> **v3 Enhancement**: Unified interface với parameter validation và metadata tracking

In [3]:
# 🏗️ Base Stream Generator Framework

@dataclass
class StreamMetadata:
    """Metadata for stream characteristics"""
    name: str
    length: int
    n_features: int
    n_classes: int
    drift_points: List[int]
    drift_type: str  # 'abrupt', 'gradual', 'recurring', 'incremental'
    noise_level: float
    description: str

class BaseStreamGenerator:
    """Base class for all stream generators with unified interface"""
    
    def __init__(self, length: int, drift_points: List[int], noise: float = 0.0):
        self.length = length
        self.drift_points = list(drift_points) if drift_points else []
        self.noise = noise
        
    def generate(self) -> Tuple[np.ndarray, np.ndarray, List[int]]:
        """Generate stream data. Returns (X, y, drift_points)"""
        raise NotImplementedError
        
    def get_metadata(self) -> StreamMetadata:
        """Get stream metadata for analysis"""
        raise NotImplementedError
        
    def validate_parameters(self):
        """Validate generator parameters"""
        assert self.length > 0, "Stream length must be positive"
        assert 0 <= self.noise <= 1, "Noise must be between 0 and 1"
        assert all(0 < dp < self.length for dp in self.drift_points), "Drift points must be within stream bounds"

print("✅ Base stream framework initialized")


✅ Base stream framework initialized


In [4]:

class SEAStream(BaseStreamGenerator):
    """
    SEA Concepts: Binary classification with abrupt threshold changes
    
    Description: X₁ + X₂ ≤ threshold determines class. Threshold changes at drift points.
    Features: 3D uniform random [0,10], only first 2 dimensions relevant
    Drift Type: Abrupt concept drift via threshold modification
    """
    
    def __init__(self, length=10000, thresholds=(7, 8, 9, 9.5), drift_points=(2500, 5000, 7500), noise=0.0):
        super().__init__(length, drift_points, noise)
        self.thresholds = thresholds
        self.validate_parameters()
        
    def validate_parameters(self):
        super().validate_parameters()
        assert len(self.thresholds) == len(self.drift_points) + 1, \
            "Number of thresholds must be drift_points + 1"
        assert all(isinstance(t, (int, float)) for t in self.thresholds), \
            "All thresholds must be numeric"

    def generate(self) -> Tuple[np.ndarray, np.ndarray, List[int]]:
        # Create timeline with thresholds
        timeline_points = [0] + self.drift_points + [self.length]
        
        def get_threshold_for_time(t):
            for k in range(len(timeline_points)-1):
                if timeline_points[k] <= t < timeline_points[k+1]:
                    return self.thresholds[k]
            return self.thresholds[-1]
        
        # Generate features: 3D uniform [0,10]
        X = np.random.rand(self.length, 3) * 10.0
        y = np.zeros(self.length, dtype=int)
        
        # Apply decision rule with time-varying threshold
        for i in range(self.length):
            threshold = get_threshold_for_time(i)
            decision_value = X[i,0] + X[i,1]  # Only first 2 features matter
            label = 0 if decision_value <= threshold else 1
            
            # Add noise if specified
            if self.noise > 0 and np.random.rand() < self.noise:
                label = 1 - label
                
            y[i] = label
            
        return X, y, self.drift_points
    
    def get_metadata(self) -> StreamMetadata:
        return StreamMetadata(
            name="SEA",
            length=self.length,
            n_features=3,
            n_classes=2,
            drift_points=self.drift_points,
            drift_type="abrupt",
            noise_level=self.noise,
            description=f"SEA concepts with thresholds {self.thresholds}, decision rule: X₁+X₂ ≤ threshold"
        )


In [5]:

class RotatingHyperplane(BaseStreamGenerator):
    """
    Rotating Hyperplane: Incremental drift via gradually rotating decision boundary
    
    Description: Decision boundary normal vector rotates continuously in 2D subspace
    Features: d-dimensional Gaussian random vectors
    Drift Type: Incremental/gradual drift with optional abrupt sign flips
    """
    
    def __init__(self, length=10000, d=10, angle_per_step=2*np.pi/20000, noise=0.0, abrupt_points=()):
        # Estimate drift points based on rotation
        estimated_drifts = [length//3, 2*length//3] if not abrupt_points else list(abrupt_points)
        super().__init__(length, estimated_drifts, noise)
        
        self.d = d
        self.angle_per_step = angle_per_step
        self.abrupt_points = set(abrupt_points)
        self.validate_parameters()
        
    def validate_parameters(self):
        super().validate_parameters()
        assert self.d >= 2, "Dimensionality must be at least 2"
        assert self.angle_per_step > 0, "Angle per step must be positive"
        
    def generate(self) -> Tuple[np.ndarray, np.ndarray, List[int]]:
        # Generate d-dimensional Gaussian features
        X = np.random.randn(self.length, self.d)
        y = np.zeros(self.length, dtype=int)
        
        angle = 0.0
        for i in range(self.length):
            # Gradually rotate the decision boundary
            angle += self.angle_per_step
            
            # Create normal vector in first 2 dimensions
            cos_a, sin_a = math.cos(angle), math.sin(angle)
            w = np.zeros(self.d)
            w[0], w[1] = cos_a, sin_a
            
            # Abrupt sign flip at specified points
            if i in self.abrupt_points:
                w = -w
                
            # Classify based on hyperplane
            decision_score = X[i].dot(w)
            label = 1 if decision_score >= 0 else 0
            
            # Add noise
            if self.noise > 0 and np.random.rand() < self.noise:
                label = 1 - label
                
            y[i] = label
            
        return X, y, self.drift_points
    
    def get_metadata(self) -> StreamMetadata:
        return StreamMetadata(
            name="RotatingHyperplane",
            length=self.length,
            n_features=self.d,
            n_classes=2,
            drift_points=self.drift_points,
            drift_type="incremental" if not self.abrupt_points else "mixed",
            noise_level=self.noise,
            description=f"Rotating hyperplane in {self.d}D, angle_step={self.angle_per_step:.6f}"
        )


In [6]:

def seven_segment_digit(bits7):
    # map 7 segments to a digit index by pattern (simplified; not unique)
    return int(sum(bits7) % 10)

class LEDStream:
    """LED generator: 24 binary attrs, 7 quan trọng. 
    - abrupt: tại các mốc drift -> hoán vị (permute) vị trí 7 bit quan trọng.
    - gradual: chuyển dần từ mapping cũ sang mới trong khoảng g_len.
    """
    def __init__(self, length=10000, mode='abrupt', drift_points=(3000, 6000, 8000), g_len=500, noise=0.05):
        self.length=length; self.mode=mode
        self.drift_points=list(drift_points); self.g_len=g_len; self.noise=noise

    def generate(self):
        d=24
        important = list(range(7))  # initial 7 important indices
        X = np.zeros((self.length, d), dtype=int)
        y = np.zeros(self.length, dtype=int)
        perm = list(range(d))
        next_perm = perm.copy()
        dp_idx=0
        def new_mapping(old_imp):
            # pick 7 new distinct indices uniformly
            cand = list(range(d))
            np.random.shuffle(cand)
            return sorted(cand[:7])
        pending = None  # for gradual

        for t in range(self.length):
            # feature generation
            X[t] = (np.random.rand(d)<0.5).astype(int)
            # gradually change mapping if needed
            if self.mode=='abrupt':
                if dp_idx < len(self.drift_points) and t==self.drift_points[dp_idx]:
                    important = new_mapping(important)
                    dp_idx+=1
            else:  # gradual
                if dp_idx < len(self.drift_points) and t==self.drift_points[dp_idx]:
                    pending = new_mapping(important)
                    start = t
                    dp_idx+=1
                if pending is not None:
                    alpha = min(1.0, (t - start)/max(1,self.g_len))
                    # probabilistically choose new mapping
                    if np.random.rand()<alpha:
                        important = pending
                        pending = None

            bits7 = X[t, important]
            lbl = seven_segment_digit(bits7)
            if self.noise>0 and np.random.rand()<self.noise:
                lbl = (lbl + np.random.randint(1,10))%10
            y[t]=lbl
        return X, y, self.drift_points


In [7]:

class InterchangingRBF:
    """RBF clusters, class labels of clusters hoán đổi tại các mốc drift (recurring/abrupt)."""
    def __init__(self, length=10000, d=10, n_centers=6, drift_points=(3000, 7000), noise=0.0):
        self.length=length; self.d=d; self.n_centers=n_centers
        self.drift_points=list(drift_points); self.noise=noise

    def generate(self):
        # init centers and class labels
        centers = np.random.randn(self.n_centers, self.d)*2.0
        labels = np.array([i%2 for i in range(self.n_centers)], dtype=int)  # binary classes
        X = np.zeros((self.length, self.d))
        y = np.zeros(self.length, dtype=int)
        dp_set = set(self.drift_points)
        for t in range(self.length):
            k = np.random.randint(0, self.n_centers)
            X[t] = centers[k] + 0.5*np.random.randn(self.d)
            lbl = labels[k]
            if self.noise>0 and np.random.rand()<self.noise:
                lbl = 1-lbl
            y[t]=lbl
            if t in dp_set:
                # swap class labels by rotating
                labels = 1 - labels  # simple invert all
        return X, y, self.drift_points


## 2) Online Learner: Gaussian Naive Bayes (incremental)

In [8]:

class OnlineGaussianNB:
    def __init__(self, n_features, n_classes=2, var_smoothing=1e-9):
        self.n_features = n_features
        self.n_classes = n_classes
        self.var_smoothing = var_smoothing
        self.counts = np.zeros(n_classes, dtype=float)
        self.means = np.zeros((n_classes, n_features), dtype=float)
        self.M2 = np.zeros((n_classes, n_features), dtype=float)
        self._eps = 1e-12

    def partial_fit(self, X, y):
        X = np.atleast_2d(X); y = np.atleast_1d(y)
        for xi, yi in zip(X, y):
            c = int(yi) if yi < self.n_classes else int(yi % self.n_classes)
            self.counts[c] += 1.0
            delta = xi - self.means[c]
            self.means[c] += delta / max(self.counts[c],1.0)
            delta2 = xi - self.means[c]
            self.M2[c] += delta * delta2

    def _vars(self):
        var = np.zeros_like(self.M2)
        for c in range(self.n_classes):
            denom = max(self.counts[c]-1.0, 1.0)
            var[c] = self.M2[c] / denom + self.var_smoothing
        return var

    def predict_proba(self, X):
        X = np.atleast_2d(X)
        var = self._vars()
        priors = (self.counts + self._eps)/(self.counts.sum() + self.n_classes*self._eps)
        logp = []
        for c in range(self.n_classes):
            # For each class c, compute log probability for all samples in X
            # var[c] and self.means[c] are 1D arrays with shape (n_features,)
            # X has shape (n_samples, n_features)
            log_var_term = -0.5 * np.sum(np.log(2*np.pi*var[c]))  # scalar
            diff_sq = ((X - self.means[c])**2) / var[c]  # shape (n_samples, n_features)
            quad_term = -0.5 * np.sum(diff_sq, axis=1)  # shape (n_samples,)
            lp = log_var_term + quad_term + np.log(priors[c]+self._eps)
            logp.append(lp)
        logp = np.vstack(logp).T
        m = np.max(logp, axis=1, keepdims=True)
        p = np.exp(logp - m); p = p/np.sum(p, axis=1, keepdims=True)
        return p

    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)


## 🚨 3) Enhanced Drift Detection Framework

> **v3 Enhancement**: Unified detector interface + River library integration + ShapeDD optimization

In [9]:
# 🏗️ Unified Drift Detector Framework

from abc import ABC, abstractmethod
from enum import Enum

class DetectorType(Enum):
    SUPERVISED = "supervised"     # Uses prediction errors
    UNSUPERVISED = "unsupervised" # Uses feature distributions
    SEMI_SUPERVISED = "semi_supervised"

@dataclass
class DetectionResult:
    """Standardized detection result"""
    detector_name: str
    timestamp: int
    is_drift: bool
    confidence: float = 0.0
    raw_statistic: float = 0.0
    metadata: Dict[str, Any] = None

class BaseDriftDetector(ABC):
    """Unified interface for all drift detectors"""
    
    def __init__(self, name: str, detector_type: DetectorType):
        self.name = name
        self.detector_type = detector_type
        self.alarms = []
        self.t = 0
        self.detection_history = []
        
    @abstractmethod
    def update(self, *args, **kwargs) -> Optional[DetectionResult]:
        """Update detector with new data. Returns detection result if drift detected."""
        pass
    
    @abstractmethod
    def reset(self):
        """Reset detector state"""
        pass
    
    def get_alarms(self) -> List[int]:
        """Get list of alarm timestamps"""
        return self.alarms.copy()
    
    def get_detection_rate(self, window_size: int) -> float:
        """Calculate recent detection rate"""
        if len(self.detection_history) < window_size:
            return 0.0
        recent = self.detection_history[-window_size:]
        return sum(1 for r in recent if r.is_drift) / len(recent)

class RiverDetectorWrapper(BaseDriftDetector):
    """Wrapper for River library detectors"""
    
    def __init__(self, river_detector, name: str):
        super().__init__(name, DetectorType.SUPERVISED)
        self.detector = river_detector
        
    def update(self, error_or_value) -> Optional[DetectionResult]:
        self.t += 1
        
        # Update the River detector
        self.detector.update(error_or_value)
        is_drift = self.detector.change_detected
        
        result = DetectionResult(
            detector_name=self.name,
            timestamp=self.t,
            is_drift=is_drift,
            confidence=1.0 if is_drift else 0.0,
            metadata={"river_detector": type(self.detector).__name__}
        )
        
        self.detection_history.append(result)
        
        if is_drift:
            self.alarms.append(self.t)
            return result
            
        return None
    
    def reset(self):
        # River detectors don't have a standard reset, so we recreate
        detector_class = type(self.detector)
        self.detector = detector_class()
        self.alarms = []
        self.t = 0
        self.detection_history = []

print("✅ Unified drift detector framework initialized")


✅ Unified drift detector framework initialized


In [10]:

class ShapeDD(BaseDriftDetector):
    """
    Enhanced ShapeDD: Kernel-based MMD detector using statistical moments
    
    Detects distribution changes via shape statistics (mean, std, skewness, kurtosis)
    in sliding reference and current windows.
    """
    
    def __init__(self, w_ref=200, w_cur=200, calib_size=1000, q=0.995, min_delay=50):
        super().__init__("ShapeDD", DetectorType.UNSUPERVISED)
        
        self.w_ref = w_ref
        self.w_cur = w_cur
        self.calib_size = calib_size
        self.q = q
        self.min_delay = min_delay
        
        self.buffer = deque(maxlen=w_ref + w_cur + 5)
        self.last_alarm_t = -10**9
        self.calib_stats = []
        self.thr = None
        
    @staticmethod
    def _moments(X):
        """Compute statistical moments: mean, std, skewness, kurtosis"""
        X = np.asarray(X)
        if X.ndim == 1:
            X = X.reshape(-1, 1)
            
        m = X.mean(axis=0)
        std = X.std(axis=0) + 1e-9
        z = (X - m) / std
        skew = np.mean(z**3, axis=0)
        kurt = np.mean(z**4, axis=0) - 3.0
        
        return m, std, skew, kurt
    
    def _shape_statistic(self, X):
        """Concatenate all statistical moments into shape vector"""
        m, s, g, k = self._moments(X)
        return np.concatenate([m, s, g, k])
    
    def update(self, x) -> Optional[DetectionResult]:
        """Update with new feature vector"""
        self.t += 1
        self.buffer.append(x)
        
        # Need sufficient data for reference and current windows
        if len(self.buffer) < (self.w_ref + self.w_cur):
            return None
            
        # Extract reference and current windows
        arr = np.array(self.buffer)
        ref_window = arr[-(self.w_ref + self.w_cur):-self.w_cur]
        cur_window = arr[-self.w_cur:]
        
        # Compute shape statistics
        s_ref = self._shape_statistic(ref_window)
        s_cur = self._shape_statistic(cur_window)
        
        # Normalized L2 distance between shape vectors
        scale = np.maximum(np.abs(s_ref), 1e-6)
        stat = np.linalg.norm((s_cur - s_ref) / scale)
        
        # Calibration phase: collect statistics
        if self.t <= self.calib_size:
            self.calib_stats.append(stat)
            return None
            
        # Set threshold after calibration
        if self.thr is None and len(self.calib_stats) > 10:
            self.thr = float(np.quantile(self.calib_stats, self.q))
            
        # Detection phase
        is_drift = (self.thr is not None and 
                   stat > self.thr and 
                   (self.t - self.last_alarm_t) >= self.min_delay)
        
        result = DetectionResult(
            detector_name=self.name,
            timestamp=self.t,
            is_drift=is_drift,
            confidence=min(stat / self.thr, 2.0) if self.thr is not None else 0.0,
            raw_statistic=stat,
            metadata={
                "threshold": self.thr,
                "calibration_size": len(self.calib_stats),
                "buffer_size": len(self.buffer)
            }
        )
        
        self.detection_history.append(result)
        
        if is_drift:
            self.last_alarm_t = self.t
            self.alarms.append(self.t)
            return result
            
        return None
    
    def reset(self):
        """Reset detector to initial state"""
        super().__init__("ShapeDD", DetectorType.UNSUPERVISED)
        self.buffer.clear()
        self.last_alarm_t = -10**9
        self.calib_stats = []
        self.thr = None

class DDM:
    def __init__(self, min_delay=50):
        self.n=0; self.p=0.0; self.pmin=float('inf'); self.smin=float('inf')
        self.alarms=[]; self.t=0; self.last_alarm_t=-10**9; self.min_delay=min_delay
    def update(self, is_error: bool):
        self.t+=1; self.n+=1; self.p += 1.0 if is_error else 0.0
        phat = self.p/self.n; s = math.sqrt(phat*(1-phat)/self.n)
        if phat + s < self.pmin + self.smin: self.pmin=phat; self.smin=s
        if phat + s >= self.pmin + 3*self.smin and (self.t - self.last_alarm_t)>=self.min_delay:
            self.alarms.append(self.t); self.last_alarm_t=self.t; return self.t
        return None

class PageHinkley:
    def __init__(self, delta=0.005, lambda_=5.0, min_delay=50):
        self.delta=delta; self.lambda_=lambda_
        self.t=0; self.mean=0.0; self.m_t=0.0; self.M_t=0.0
        self.alarms=[]; self.last_alarm_t=-10**9; self.min_delay=min_delay
    def update(self, value):
        self.t+=1
        self.mean = self.mean + (value - self.mean)/self.t
        self.m_t += (value - self.mean - self.delta)
        self.M_t = min(self.M_t, self.m_t)
        if (self.m_t - self.M_t) > self.lambda_ and (self.t - self.last_alarm_t)>=self.min_delay:
            self.alarms.append(self.t); self.m_t=0.0; self.M_t=0.0; self.last_alarm_t=self.t; return self.t
        return None

class ADWIN:
    """Simple ADWIN-like detector on binary error stream.
    Maintains a variable-length window and checks all cut points for mean change with Hoeffding-like bound.
    """
    def __init__(self, delta=0.002, min_window=50, min_delay=50):
        self.delta=delta; self.min_window=min_window
        self.win=deque(); self.alarms=[]; self.t=0; self.last_alarm_t=-10**9; self.min_delay=min_delay
    def update(self, value):
        self.t+=1; self.win.append(value)
        if len(self.win) < self.min_window: return None
        changed=False
        n=len(self.win); arr=np.array(self.win, dtype=float)
        mu = arr.mean()
        # check a few candidate cuts (not all for speed)
        for k in np.linspace(self.min_window//2, n-self.min_window//2, num=10, dtype=int):
            left = arr[:k]; right = arr[k:]
            mu1, mu2 = left.mean(), right.mean()
            eps = math.sqrt( (1/(2*k)) * math.log(4/self.delta) ) + math.sqrt( (1/(2*(n-k))) * math.log(4/self.delta) )
            if abs(mu1 - mu2) > eps:
                changed=True; break
        if changed and (self.t - self.last_alarm_t)>=self.min_delay:
            # shrink window by dropping older half
            for _ in range(len(self.win)//2): self.win.popleft()
            self.alarms.append(self.t); self.last_alarm_t=self.t; return self.t
        return None

class MDDM:
    """McDiarmid Drift Detection (simplified): compare weighted averages in two halves of a sliding window."""
    def __init__(self, W=400, delta=0.002, min_delay=50):
        self.W=W; self.delta=delta; self.buf=deque(maxlen=W)
        self.alarms=[]; self.t=0; self.last_alarm_t=-10**9; self.min_delay=min_delay
    def update(self, value):
        self.t+=1; self.buf.append(float(value))
        if len(self.buf)<self.W: return None
        arr=np.array(self.buf); k=self.W//2
        left, right = arr[:k], arr[k:]
        # weights increasing (MDDM-A style)
        w = np.arange(1, k+1, dtype=float); w/=w.sum()
        mu1 = np.sum(left * w); mu2 = np.sum(right * w)
        # McDiarmid bound with weights: sum c_i^2 where c_i are weights' bounds in [0,1]
        Ci2 = np.sum((w)**2)
        eps = math.sqrt(0.5*Ci2*math.log(2.0/self.delta))
        if (mu2 - mu1) > eps and (self.t - self.last_alarm_t)>=self.min_delay:
            self.alarms.append(self.t); self.last_alarm_t=self.t; return self.t
        return None

class FHDDM:
    """Hoeffding Drift Detection on sliding window of correctness (1-correct,0-wrong or vice versa)."""
    def __init__(self, W=500, delta=0.002, min_delay=50):
        self.W=W; self.delta=delta; self.buf=deque(maxlen=W)
        self.mu_max=0.0; self.alarms=[]; self.t=0; self.last_alarm_t=-10**9; self.min_delay=min_delay
    def update(self, correct01):
        self.t+=1; self.buf.append(float(correct01))
        if len(self.buf)<self.W: return None
        mu = np.mean(self.buf)
        self.mu_max = max(self.mu_max, mu)
        eps = math.sqrt((1/(2*self.W))*math.log(1/self.delta))
        if (self.mu_max - mu) >= eps and (self.t - self.last_alarm_t)>=self.min_delay:
            self.alarms.append(self.t); self.mu_max = mu  # reset baseline
            self.last_alarm_t=self.t; return self.t
        return None

class FHDDMS:
    """Stacking FHDDM: short & long windows; alarm if any triggers."""
    def __init__(self, W_short=100, W_long=500, delta=0.002, min_delay=50):
        self.short = FHDDM(W=W_short, delta=delta, min_delay=min_delay)
        self.long = FHDDM(W=W_long,  delta=delta, min_delay=min_delay)
        self.alarms=[]; self.t=0
    def update(self, correct01):
        self.t+=1
        a1 = self.short.update(correct01)
        a2 = self.long.update(correct01)
        fired = False
        for a in (a1,a2):
            if a is not None: fired=True
        if fired:
            self.alarms.append(self.t)
            return self.t
        return None


## 4) Evaluation (prequential)

In [11]:

@dataclass
class RunResult:
    stream: str
    name: str
    accuracy: float
    macro_f1: float
    n_det: int
    false_alarms: int
    mean_delay: Optional[float]
    delays: List[int]
    alarms: List[int]
    drift_points: List[int]
    runtime_s: float

def compute_delays(alarms, drift_points, tol=200):
    drift_points = list(drift_points); alarms = sorted(alarms); used=set(); delays=[]
    for dp in drift_points:
        cand = [a for a in alarms if a>=dp and (a-dp)<=tol and a not in used]
        if cand:
            a=cand[0]; used.add(a); delays.append(a-dp)
        else:
            delays.append(np.nan)
    good=set()
    for dp in drift_points:
        good.update([a for a in alarms if a>=dp and (a-dp)<=tol])
    false_alarms = len([a for a in alarms if a not in good])
    md = np.nanmean(delays) if len(delays)>0 else np.nan
    return delays, false_alarms, md

def update_confusion(cm, y_true, y_pred, n_classes):
    cm[y_true, y_pred] += 1
    return cm

def metrics_from_cm(cm):
    # Accuracy
    acc = np.trace(cm)/np.sum(cm) if cm.sum()>0 else 0.0
    # Macro F1
    K = cm.shape[0]
    f1s=[]
    for k in range(K):
        TP = cm[k,k]
        FP = cm[:,k].sum() - TP
        FN = cm[k,:].sum() - TP
        prec = TP/(TP+FP) if (TP+FP)>0 else 0.0
        rec  = TP/(TP+FN) if (TP+FN)>0 else 0.0
        f1 = 2*prec*rec/(prec+rec) if (prec+rec)>0 else 0.0
        f1s.append(f1)
    macro_f1 = float(np.mean(f1s)) if len(f1s)>0 else 0.0
    return acc, macro_f1

def run_stream_experiment(stream_name, X, y, drift_points, detectors, learner):
    t0 = time.time()
    n = len(y)
    n_classes = int(np.max(y))+1
    cm = np.zeros((n_classes, n_classes), dtype=int)
    for i in range(n):
        xi = X[i]; yi = y[i]
        yhat = learner.predict(xi)[0] if learner.counts.sum()>0 else random.randint(0, n_classes-1)
        cm = update_confusion(cm, yi, yhat, n_classes)
        err = 0 if yhat==yi else 1
        # feed detectors
        for name, det in detectors.items():
            if isinstance(det, ShapeDD):
                det.update(xi)
            elif isinstance(det, (DDM, ADWIN, MDDM, FHDDM, FHDDMS, PageHinkley)):
                # For FHDDM(S), use correctness (1=correct)
                if isinstance(det, (FHDDM, FHDDMS)):
                    det.update(1-err==1)  # correct01
                else:
                    det.update(err)
        learner.partial_fit(xi, yi)
    # summarize per detector
    acc, macro_f1 = metrics_from_cm(cm)
    results=[]
    for name, det in detectors.items():
        alarms = getattr(det, 'alarms', [])
        delays, fa, md = compute_delays(alarms, drift_points)
        results.append(RunResult(
            stream=stream_name, name=name, accuracy=acc, macro_f1=macro_f1,
            n_det=len(alarms), false_alarms=fa, mean_delay=md, delays=delays,
            alarms=alarms, drift_points=drift_points, runtime_s=time.time()-t0
        ))
    return results


## 5) Chạy thử nghiệm (5 streams)

In [12]:
# 🏭 Detector Factory & Configuration

def create_detector_suite(min_delay=100) -> Dict[str, BaseDriftDetector]:
    """Create a comprehensive suite of drift detectors"""
    detectors = {}
    
    # 1. Custom ShapeDD
    detectors["ShapeDD"] = ShapeDD(
        w_ref=200, w_cur=200, calib_size=1000, 
        q=0.995, min_delay=min_delay
    )
    
    # 2. Traditional methods (existing implementations)
    detectors["DDM"] = DDM(min_delay=min_delay)
    detectors["PageHinkley"] = PageHinkley(
        delta=0.001, lambda_=10.0, min_delay=min_delay
    )
    detectors["ADWIN"] = ADWIN(
        delta=0.002, min_window=80, min_delay=min_delay
    )
    detectors["MDDM"] = MDDM(
        W=400, delta=0.005, min_delay=min_delay
    )
    detectors["FHDDM"] = FHDDM(
        W=500, delta=0.002, min_delay=min_delay
    )
    detectors["FHDDMS"] = FHDDMS(
        W_short=100, W_long=500, delta=0.002, min_delay=min_delay
    )
    
    # 3. River library detectors (if available)
    if RIVER_AVAILABLE:
        detectors["River_DDM"] = RiverDetectorWrapper(
            river_drift.DDM(), "River_DDM"
        )
        detectors["River_EDDM"] = RiverDetectorWrapper(
            river_drift.EDDM(), "River_EDDM"
        )
        detectors["River_ADWIN"] = RiverDetectorWrapper(
            river_drift.ADWIN(), "River_ADWIN"
        )
        detectors["River_HDDM_A"] = RiverDetectorWrapper(
            river_drift.HDDM_A(), "River_HDDM_A"
        )
        detectors["River_HDDM_W"] = RiverDetectorWrapper(
            river_drift.HDDM_W(), "River_HDDM_W"
        )
        detectors["River_KSWIN"] = RiverDetectorWrapper(
            river_drift.KSWIN(), "River_KSWIN"
        )
        detectors["River_PageHinkley"] = RiverDetectorWrapper(
            river_drift.PageHinkley(), "River_PageHinkley"
        )
        print(f"✅ Created {len(detectors)} detectors (including {len([k for k in detectors if 'River' in k])} River detectors)")
    else:
        print(f"✅ Created {len(detectors)} detectors (River library not available)")
    
    return detectors

# Test detector creation
test_detectors = create_detector_suite()
print(f"📋 Available detectors: {list(test_detectors.keys())}")


✅ Created 7 detectors (River library not available)
📋 Available detectors: ['ShapeDD', 'DDM', 'PageHinkley', 'ADWIN', 'MDDM', 'FHDDM', 'FHDDMS']


In [13]:

# 🚀 5) Enhanced Benchmark Execution

## Stream Configuration
N = 15000  # Stream length
streams = []

print("🌊 Generating synthetic data streams...")

# Create enhanced stream generators
generators = [
    SEAStream(length=N, thresholds=(7.0, 8.0, 9.0, 9.5), 
              drift_points=(4000, 8000, 12000), noise=0.01),
    RotatingHyperplane(length=N, d=10, angle_per_step=2*np.pi/(N//2), noise=0.01),
    LEDStream(length=N, mode='abrupt', drift_points=(3000, 7000, 12000), 
              g_len=0, noise=0.02),
    LEDStream(length=N, mode='gradual', drift_points=(4000, 9000), 
              g_len=800, noise=0.02),
    InterchangingRBF(length=N, d=10, n_centers=6, 
                     drift_points=(5000, 10000), noise=0.01)
]

# Generate streams with metadata
stream_data = []
for generator in generators:
    X, y, drift_points = generator.generate()
    metadata = generator.get_metadata()
    stream_data.append({
        'name': metadata.name,
        'X': X,
        'y': y, 
        'drift_points': drift_points,
        'metadata': metadata
    })
    print(f"✅ {metadata.name}: {len(X)} samples, {metadata.n_features}D, "
          f"{len(drift_points)} drifts at {drift_points}")

print(f"📊 Generated {len(stream_data)} data streams ready for evaluation")

# Results storage
benchmark_results = []
detailed_results = {}

for sname, Xs, ys, dps in streams:
    print(f"== Running on {sname} ==")
    n_classes = int(np.max(ys))+1
    dets = {
        "ShapeDD": ShapeDD(w_ref=200, w_cur=200, calib_size=1000, q=0.995, min_delay=100),
        "DDM": DDM(min_delay=100),
        "PageHinkley": PageHinkley(delta=0.001, lambda_=10.0, min_delay=100),
        "ADWIN": ADWIN(delta=0.002, min_window=80, min_delay=100),
        "MDDM": MDDM(W=400, delta=0.005, min_delay=100),
        "FHDDM": FHDDM(W=500, delta=0.002, min_delay=100),
        "FHDDMS": FHDDMS(W_short=100, W_long=500, delta=0.002, min_delay=100),
    }
    learner = OnlineGaussianNB(n_features=Xs.shape[1], n_classes=n_classes)
    res = run_stream_experiment(sname, Xs, ys, dps, dets, learner)
    for r in res:
        all_rows.append({
            "stream": r.stream,
            "detector": r.name,
            "accuracy": r.accuracy,
            "macro_f1": r.macro_f1,
            "n_detections": r.n_det,
            "false_alarms": r.false_alarms,
            "mean_delay": r.mean_delay,
            "runtime_s": r.runtime_s
        })
        all_details[(sname, r.name)] = r

df = pd.DataFrame(all_rows)
df


🌊 Generating synthetic data streams...
✅ SEA: 15000 samples, 3D, 3 drifts at [4000, 8000, 12000]
✅ RotatingHyperplane: 15000 samples, 10D, 2 drifts at [5000, 10000]


AttributeError: 'LEDStream' object has no attribute 'get_metadata'

In [ ]:
# 📊 Enhanced Evaluation Framework

@dataclass
class EnhancedRunResult:
    """Comprehensive evaluation results"""
    stream_name: str
    detector_name: str
    detector_type: str
    
    # Classification metrics
    accuracy: float
    macro_f1: float
    precision: float
    recall: float
    
    # Detection quality
    n_detections: int
    true_positives: int
    false_alarms: int
    false_negatives: int
    
    # Timing metrics
    delays: List[float]
    mean_delay: float
    median_delay: float
    delay_std: float
    
    # Advanced metrics
    beta_score: float
    alarm_rate: float
    f1_at_ar: float
    accuracy_at_ar: float
    
    # Metadata
    runtime_s: float
    drift_points: List[int]
    alarm_timestamps: List[int]
    stream_metadata: StreamMetadata

def enhanced_delay_analysis(alarms: List[int], drift_points: List[int], 
                          tolerance: int = 500) -> Dict[str, Any]:
    """Enhanced delay analysis with statistical measures"""
    if not drift_points:
        return {
            'delays': [], 'tp': 0, 'fp': len(alarms), 'fn': 0,
            'mean_delay': np.nan, 'median_delay': np.nan, 'delay_std': np.nan
        }
    
    alarms = sorted(alarms)
    drift_points = sorted(drift_points)
    
    # Match alarms to drift points
    used_alarms = set()
    delays = []
    
    for dp in drift_points:
        # Find first unused alarm after drift point within tolerance
        candidates = [a for a in alarms 
                     if a >= dp and (a - dp) <= tolerance and a not in used_alarms]
        if candidates:
            alarm = min(candidates)
            delays.append(alarm - dp)
            used_alarms.add(alarm)
    
    tp = len(delays)
    fp = len([a for a in alarms if a not in used_alarms])
    fn = len(drift_points) - tp
    
    # Statistical measures
    if delays:
        mean_delay = np.mean(delays)
        median_delay = np.median(delays)
        delay_std = np.std(delays)
    else:
        mean_delay = median_delay = delay_std = np.nan
    
    return {
        'delays': delays,
        'tp': tp, 'fp': fp, 'fn': fn,
        'mean_delay': mean_delay,
        'median_delay': median_delay, 
        'delay_std': delay_std
    }

def compute_advanced_metrics(acc: float, f1: float, n_alarms: int, 
                           n_samples: int, tp: int, fp: int, fn: int,
                           beta: float = 0.5, lambda_ar: float = 0.01) -> Dict[str, float]:
    """Compute advanced evaluation metrics"""
    # Alarm rate per 10k samples
    alarm_rate = (n_alarms / n_samples) * 10000
    
    # F1@AR and Accuracy@AR
    f1_at_ar = f1 - lambda_ar * alarm_rate
    acc_at_ar = acc - lambda_ar * alarm_rate
    
    # β-score (balanced precision-recall for detection)
    p = tp + fn  # Total true drifts
    beta_score = tp / (p + beta * fp) if (p + beta * fp) > 0 else 0.0
    
    return {
        'alarm_rate': alarm_rate,
        'f1_at_ar': f1_at_ar,
        'accuracy_at_ar': acc_at_ar,
        'beta_score': beta_score
    }

print("✅ Enhanced evaluation framework ready")


In [ ]:
# 🚀 Main Benchmark Execution Function

def run_enhanced_benchmark(stream_data: Dict, detectors: Dict[str, BaseDriftDetector], 
                          use_river_classifier: bool = True) -> List[EnhancedRunResult]:
    """
    Run enhanced benchmark with comprehensive evaluation
    """
    results = []
    
    # Create classifier
    if use_river_classifier and RIVER_AVAILABLE:
        classifier_factory = lambda: tree.HoeffdingTreeClassifier()
        print("🌳 Using River HoeffdingTreeClassifier")
    else:
        # Use existing OnlineGaussianNB as fallback
        classifier_factory = lambda: OnlineGaussianNB(
            n_features=stream_data['X'].shape[1], 
            n_classes=int(np.max(stream_data['y'])) + 1
        )
        print("📊 Using OnlineGaussianNB classifier")
    
    X, y = stream_data['X'], stream_data['y']
    drift_points = stream_data['drift_points']
    metadata = stream_data['metadata']
    
    print(f"\\n🎯 Processing {metadata.name} stream...")
    print(f"   📏 Length: {len(X)}, Features: {X.shape[1]}, Classes: {metadata.n_classes}")
    print(f"   🌊 Drift points: {drift_points}")
    
    for det_name, detector in detectors.items():
        start_time = time.time()
        
        # Reset detector and create fresh classifier
        detector.reset()
        classifier = classifier_factory()
        
        # Classification tracking
        predictions = []
        true_labels = []
        
        print(f"     🔍 Testing {det_name}...", end=' ')
        
        # Prequential evaluation loop
        for i in range(len(X)):
            xi, yi = X[i], y[i]
            
            # Predict with current model
            if RIVER_AVAILABLE and use_river_classifier:
                y_pred = classifier.predict_one(xi if isinstance(xi, dict) else {f'f{j}': xi[j] for j in range(len(xi))})
                if y_pred is None:
                    y_pred = np.random.randint(0, metadata.n_classes)
            else:
                if hasattr(classifier, 'predict') and classifier.counts.sum() > 0:
                    y_pred = classifier.predict(xi.reshape(1, -1))[0]
                else:
                    y_pred = np.random.randint(0, metadata.n_classes)
            
            predictions.append(y_pred)
            true_labels.append(yi)
            
            # Update drift detector
            error = 1 if y_pred != yi else 0
            
            if isinstance(detector, ShapeDD):
                # ShapeDD uses feature vectors
                detector.update(xi)
            elif isinstance(detector, RiverDetectorWrapper):
                # River detectors use errors or probabilities
                if 'KSWIN' in det_name:
                    # KSWIN needs probability, use random for simplicity
                    detector.update(np.random.random())
                else:
                    detector.update(error)
            else:
                # Traditional detectors use errors
                if hasattr(detector, 'update'):
                    if det_name in ['FHDDM', 'FHDDMS']:
                        detector.update(1 - error)  # Correctness for FHDDM
                    else:
                        detector.update(error)
            
            # Update classifier
            if RIVER_AVAILABLE and use_river_classifier:
                classifier.learn_one(
                    xi if isinstance(xi, dict) else {f'f{j}': xi[j] for j in range(len(xi))}, 
                    yi
                )
            else:
                classifier.partial_fit(xi.reshape(1, -1), [yi])
        
        runtime = time.time() - start_time
        
        # Compute classification metrics
        acc = accuracy_score(true_labels, predictions)
        f1 = f1_score(true_labels, predictions, average='macro', zero_division=0)
        precision = precision_score(true_labels, predictions, average='macro', zero_division=0)
        recall = recall_score(true_labels, predictions, average='macro', zero_division=0)
        
        # Get detection results
        alarms = detector.get_alarms()
        delay_analysis = enhanced_delay_analysis(alarms, drift_points)
        
        # Compute advanced metrics
        advanced = compute_advanced_metrics(
            acc, f1, len(alarms), len(X),
            delay_analysis['tp'], delay_analysis['fp'], delay_analysis['fn']
        )
        
        # Create comprehensive result
        result = EnhancedRunResult(
            stream_name=metadata.name,
            detector_name=det_name,
            detector_type=detector.detector_type.value if hasattr(detector, 'detector_type') else 'unknown',
            accuracy=acc,
            macro_f1=f1,
            precision=precision,
            recall=recall,
            n_detections=len(alarms),
            true_positives=delay_analysis['tp'],
            false_alarms=delay_analysis['fp'],
            false_negatives=delay_analysis['fn'],
            delays=delay_analysis['delays'],
            mean_delay=delay_analysis['mean_delay'],
            median_delay=delay_analysis['median_delay'],
            delay_std=delay_analysis['delay_std'],
            beta_score=advanced['beta_score'],
            alarm_rate=advanced['alarm_rate'],
            f1_at_ar=advanced['f1_at_ar'],
            accuracy_at_ar=advanced['accuracy_at_ar'],
            runtime_s=runtime,
            drift_points=drift_points,
            alarm_timestamps=alarms,
            stream_metadata=metadata
        )
        
        results.append(result)
        print(f"✅ Acc:{acc:.3f}, F1:{f1:.3f}, Detections:{len(alarms)}, Runtime:{runtime:.2f}s")
    
    return results

print("🚀 Enhanced benchmark function ready")


In [ ]:
# 🎯 Execute Enhanced Benchmark

# Create detector suite
detectors = create_detector_suite(min_delay=100)

# Run benchmark on all streams
all_results = []

for stream in stream_data:
    print(f"\\n{'='*60}")
    print(f"🌊 BENCHMARKING: {stream['name'].upper()}")
    print(f"{'='*60}")
    
    stream_results = run_enhanced_benchmark(stream, detectors)
    all_results.extend(stream_results)

print(f"\\n🎉 Benchmark Complete! Processed {len(all_results)} detector-stream combinations")

# Convert to DataFrame for analysis
results_df = pd.DataFrame([
    {
        'Stream': r.stream_name,
        'Detector': r.detector_name,
        'Type': r.detector_type,
        'Accuracy': r.accuracy,
        'Macro_F1': r.macro_f1,
        'Precision': r.precision,
        'Recall': r.recall,
        'Detections': r.n_detections,
        'True_Positives': r.true_positives,
        'False_Alarms': r.false_alarms,
        'False_Negatives': r.false_negatives,
        'Mean_Delay': r.mean_delay,
        'Median_Delay': r.median_delay,
        'Delay_Std': r.delay_std,
        'Beta_Score': r.beta_score,
        'Alarm_Rate': r.alarm_rate,
        'F1@AR': r.f1_at_ar,
        'Accuracy@AR': r.accuracy_at_ar,
        'Runtime_s': r.runtime_s,
        'Drift_Points': str(r.drift_points),
        'Alarm_Times': str(r.alarm_timestamps)
    }
    for r in all_results
])

print(f"\\n📊 Results DataFrame shape: {results_df.shape}")
print("\\n🔝 Top 5 detectors by F1@AR:")
top_f1ar = results_df.nlargest(5, 'F1@AR')[['Stream', 'Detector', 'F1@AR', 'Accuracy', 'Beta_Score']]
print(top_f1ar.to_string(index=False))

results_df.head(10)


In [ ]:
# 📈 Advanced Visualization & Analysis

def create_comprehensive_plots(results_df):
    """Create comprehensive visualization suite"""
    
    # Set style
    plt.style.use('default')
    sns.set_palette("husl")
    
    fig = plt.figure(figsize=(20, 15))
    
    # 1. Performance heatmap
    plt.subplot(3, 3, 1)
    pivot_f1 = results_df.pivot(index='Detector', columns='Stream', values='F1@AR')
    sns.heatmap(pivot_f1, annot=True, fmt='.3f', cmap='RdYlGn', 
                cbar_kws={'label': 'F1@AR Score'})
    plt.title('🎯 F1@AR Performance Heatmap')
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    
    # 2. Detection quality scatter
    plt.subplot(3, 3, 2)
    scatter = plt.scatter(results_df['False_Alarms'], results_df['Mean_Delay'], 
                         c=results_df['Beta_Score'], s=results_df['True_Positives']*20,
                         cmap='viridis', alpha=0.7)
    plt.xlabel('False Alarms')
    plt.ylabel('Mean Detection Delay')
    plt.title('🎯 Detection Quality (size=TPs, color=β-score)')
    plt.colorbar(scatter, label='β-score')
    
    # 3. Accuracy vs Alarm Rate
    plt.subplot(3, 3, 3)
    for stream in results_df['Stream'].unique():
        stream_data = results_df[results_df['Stream'] == stream]
        plt.scatter(stream_data['Alarm_Rate'], stream_data['Accuracy'], 
                   label=stream, alpha=0.7, s=60)
    plt.xlabel('Alarm Rate (per 10k samples)')
    plt.ylabel('Accuracy')
    plt.title('📊 Accuracy vs Alarm Rate')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # 4. Runtime comparison
    plt.subplot(3, 3, 4)
    runtime_by_detector = results_df.groupby('Detector')['Runtime_s'].mean().sort_values()
    runtime_by_detector.plot(kind='barh', color='skyblue')
    plt.xlabel('Mean Runtime (seconds)')
    plt.title('⏱️ Runtime Performance')
    plt.tight_layout()
    
    # 5. Detection delay distribution
    plt.subplot(3, 3, 5)
    delay_data = []
    delay_labels = []
    for detector in results_df['Detector'].unique()[:8]:  # Top 8 for clarity
        detector_results = results_df[results_df['Detector'] == detector]
        delays = []
        for _, row in detector_results.iterrows():
            if not pd.isna(row['Mean_Delay']):
                delays.extend(row['Mean_Delay'] if isinstance(row['Mean_Delay'], list) else [row['Mean_Delay']])
        if delays:
            delay_data.append(delays)
            delay_labels.append(detector)
    
    if delay_data:
        plt.boxplot(delay_data, labels=delay_labels)
        plt.xticks(rotation=45)
        plt.ylabel('Detection Delay')
        plt.title('📦 Detection Delay Distribution')
    
    # 6. Stream difficulty ranking
    plt.subplot(3, 3, 6)
    stream_difficulty = results_df.groupby('Stream').agg({
        'Accuracy': 'mean',
        'Mean_Delay': 'mean',
        'False_Alarms': 'mean'
    }).reset_index()
    
    # Normalize metrics (lower is better for difficulty score)
    stream_difficulty['Difficulty'] = (
        (1 - stream_difficulty['Accuracy']) + 
        stream_difficulty['Mean_Delay'].fillna(0) / 1000 +
        stream_difficulty['False_Alarms'] / 10
    )
    
    stream_difficulty = stream_difficulty.sort_values('Difficulty')
    plt.barh(stream_difficulty['Stream'], stream_difficulty['Difficulty'], color='coral')
    plt.xlabel('Difficulty Score (lower=easier)')
    plt.title('🏔️ Stream Difficulty Ranking')
    
    # 7. Advanced metrics comparison
    plt.subplot(3, 3, 7)
    metrics_comparison = results_df.groupby('Detector')[['F1@AR', 'Accuracy@AR', 'Beta_Score']].mean()
    metrics_comparison.plot(kind='bar', ax=plt.gca())
    plt.xticks(rotation=45)
    plt.ylabel('Score')
    plt.title('📊 Advanced Metrics Comparison')
    plt.legend()
    
    # 8. Detector type performance
    plt.subplot(3, 3, 8)
    type_performance = results_df.groupby('Type')[['Accuracy', 'F1@AR', 'Beta_Score']].mean()
    type_performance.plot(kind='bar', ax=plt.gca())
    plt.xticks(rotation=45)
    plt.ylabel('Score')
    plt.title('🔧 Performance by Detector Type')
    plt.legend()
    
    # 9. Global ranking
    plt.subplot(3, 3, 9)
    # Compute global score (normalized combination)
    for col in ['F1@AR', 'Accuracy@AR', 'Beta_Score']:
        col_min, col_max = results_df[col].min(), results_df[col].max()
        if col_max > col_min:
            results_df[f'{col}_norm'] = (results_df[col] - col_min) / (col_max - col_min)
        else:
            results_df[f'{col}_norm'] = 0.5
    
    results_df['Global_Score'] = (
        results_df['F1@AR_norm'] * 0.4 + 
        results_df['Accuracy@AR_norm'] * 0.3 + 
        results_df['Beta_Score_norm'] * 0.3
    )
    
    global_ranking = results_df.groupby('Detector')['Global_Score'].mean().sort_values(ascending=False)[:10]
    global_ranking.plot(kind='barh', color='gold')
    plt.xlabel('Global Score')
    plt.title('🏆 Top 10 Global Detector Ranking')
    
    plt.tight_layout()
    plt.show()
    
    return results_df

# Create visualizations
enhanced_results_df = create_comprehensive_plots(results_df)

print("\\n🏆 FINAL RANKINGS:")
print("="*50)

# Top detectors by different criteria
print("\\n🎯 Best F1@AR Performance:")
top_f1ar = enhanced_results_df.groupby('Detector')['F1@AR'].mean().sort_values(ascending=False).head(5)
for i, (detector, score) in enumerate(top_f1ar.items(), 1):
    print(f"{i}. {detector}: {score:.4f}")

print("\\n🔍 Best Detection Quality (β-score):")
top_beta = enhanced_results_df.groupby('Detector')['Beta_Score'].mean().sort_values(ascending=False).head(5)
for i, (detector, score) in enumerate(top_beta.items(), 1):
    print(f"{i}. {detector}: {score:.4f}")

print("\\n⚡ Fastest Detectors:")
top_speed = enhanced_results_df.groupby('Detector')['Runtime_s'].mean().sort_values().head(5)
for i, (detector, time) in enumerate(top_speed.items(), 1):
    print(f"{i}. {detector}: {time:.3f}s")

print("\\n🏆 Overall Champions (Global Score):")
if 'Global_Score' in enhanced_results_df.columns:
    top_global = enhanced_results_df.groupby('Detector')['Global_Score'].mean().sort_values(ascending=False).head(5)
    for i, (detector, score) in enumerate(top_global.items(), 1):
        print(f"{i}. {detector}: {score:.4f}")

print("\\n" + "="*50)


In [ ]:
# 💾 Export Results & Summary Report

from datetime import datetime
import json

def export_comprehensive_results(results_df, output_dir="./results"):
    """Export comprehensive results in multiple formats"""
    
    # Create output directory
    Path(output_dir).mkdir(exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 1. Full results CSV
    full_results_path = f"{output_dir}/concept_drift_benchmark_v3_{timestamp}.csv"
    results_df.to_csv(full_results_path, index=False)
    print(f"📄 Full results exported: {full_results_path}")
    
    # 2. Summary statistics
    summary_stats = {}
    
    # Overall performance by detector
    detector_summary = results_df.groupby('Detector').agg({
        'Accuracy': ['mean', 'std'],
        'Macro_F1': ['mean', 'std'], 
        'F1@AR': ['mean', 'std'],
        'Beta_Score': ['mean', 'std'],
        'Mean_Delay': ['mean', 'std'],
        'False_Alarms': ['mean', 'std'],
        'Runtime_s': ['mean', 'std']
    }).round(4)
    
    detector_summary.columns = ['_'.join(col).strip() for col in detector_summary.columns]
    summary_path = f"{output_dir}/detector_summary_{timestamp}.csv"
    detector_summary.to_csv(summary_path)
    print(f"📊 Detector summary exported: {summary_path}")
    
    # Stream performance summary
    stream_summary = results_df.groupby('Stream').agg({
        'Accuracy': ['mean', 'std'],
        'F1@AR': ['mean', 'std'],
        'Beta_Score': ['mean', 'std'],
        'False_Alarms': ['mean', 'std']
    }).round(4)
    
    stream_summary.columns = ['_'.join(col).strip() for col in stream_summary.columns]
    stream_path = f"{output_dir}/stream_summary_{timestamp}.csv"
    stream_summary.to_csv(stream_path)
    print(f"🌊 Stream summary exported: {stream_path}")
    
    # 3. Best performers report
    best_performers = {
        'timestamp': timestamp,
        'total_experiments': len(results_df),
        'detectors_tested': results_df['Detector'].nunique(),
        'streams_tested': results_df['Stream'].nunique(),
        'best_f1_ar': {
            'detector': results_df.loc[results_df['F1@AR'].idxmax(), 'Detector'],
            'score': float(results_df['F1@AR'].max()),
            'stream': results_df.loc[results_df['F1@AR'].idxmax(), 'Stream']
        },
        'best_beta_score': {
            'detector': results_df.loc[results_df['Beta_Score'].idxmax(), 'Detector'],
            'score': float(results_df['Beta_Score'].max()),
            'stream': results_df.loc[results_df['Beta_Score'].idxmax(), 'Stream']
        },
        'fastest_detector': {
            'detector': results_df.loc[results_df['Runtime_s'].idxmin(), 'Detector'],
            'time': float(results_df['Runtime_s'].min()),
            'stream': results_df.loc[results_df['Runtime_s'].idxmin(), 'Stream']
        }
    }
    
    # Add rankings
    if 'Global_Score' in results_df.columns:
        global_ranking = results_df.groupby('Detector')['Global_Score'].mean().sort_values(ascending=False)
        best_performers['global_ranking'] = global_ranking.head(10).to_dict()
    
    # Export as JSON
    json_path = f"{output_dir}/best_performers_{timestamp}.json"
    with open(json_path, 'w') as f:
        json.dump(best_performers, f, indent=2)
    print(f"🏆 Best performers report: {json_path}")
    
    # 4. LaTeX table for papers
    latex_summary = results_df.groupby('Detector')[['Accuracy', 'Macro_F1', 'F1@AR', 'Beta_Score', 'Mean_Delay']].mean()
    latex_table = latex_summary.round(3).to_latex(float_format="%.3f")
    
    latex_path = f"{output_dir}/latex_table_{timestamp}.tex"
    with open(latex_path, 'w') as f:
        f.write("% Enhanced Concept Drift Benchmark Results\\n")
        f.write("% Generated by ConceptDrift_Baseline_v3\\n\\n")
        f.write(latex_table)
    print(f"📝 LaTeX table exported: {latex_path}")
    
    # 5. Create README with experiment details
    readme_content = f"""# Concept Drift Benchmark Results v3
    
## Experiment Details
- **Timestamp**: {timestamp}
- **Total Experiments**: {len(results_df)}
- **Detectors Tested**: {results_df['Detector'].nunique()}
- **Streams Tested**: {results_df['Stream'].nunique()}
- **Framework**: Enhanced Baseline v3 with unified interface

## Files Generated
- `concept_drift_benchmark_v3_{timestamp}.csv`: Full experimental results
- `detector_summary_{timestamp}.csv`: Performance statistics by detector
- `stream_summary_{timestamp}.csv`: Performance statistics by stream  
- `best_performers_{timestamp}.json`: Top performers and rankings
- `latex_table_{timestamp}.tex`: LaTeX formatted results table

## Top Performers

### Best F1@AR Score
- **Detector**: {best_performers['best_f1_ar']['detector']}
- **Score**: {best_performers['best_f1_ar']['score']:.4f}
- **Stream**: {best_performers['best_f1_ar']['stream']}

### Best Detection Quality (β-score)  
- **Detector**: {best_performers['best_beta_score']['detector']}
- **Score**: {best_performers['best_beta_score']['score']:.4f}
- **Stream**: {best_performers['best_beta_score']['stream']}

### Fastest Detection
- **Detector**: {best_performers['fastest_detector']['detector']}
- **Runtime**: {best_performers['fastest_detector']['time']:.3f}s
- **Stream**: {best_performers['fastest_detector']['stream']}

## Metrics Explanation
- **F1@AR**: F1-score penalized by alarm rate (F1 - λ×AR)
- **β-score**: Balanced detection precision-recall measure
- **Mean_Delay**: Average detection delay in samples
- **Accuracy@AR**: Accuracy penalized by alarm rate

Generated by Enhanced Concept Drift Benchmark v3
"""
    
    readme_path = f"{output_dir}/README_{timestamp}.md"
    with open(readme_path, 'w') as f:
        f.write(readme_content)
    print(f"📋 Experiment README: {readme_path}")
    
    print(f"\\n✅ All results exported to: {Path(output_dir).absolute()}")
    return output_dir

# Export comprehensive results
export_dir = export_comprehensive_results(enhanced_results_df)

print(f"""
🎉 ENHANCED CONCEPT DRIFT BENCHMARK v3 - COMPLETE! 

📊 **Summary Statistics:**
   • {len(enhanced_results_df)} total experiments
   • {enhanced_results_df['Detector'].nunique()} drift detectors tested
   • {enhanced_results_df['Stream'].nunique()} data streams evaluated
   • {enhanced_results_df['Runtime_s'].sum():.2f}s total runtime

🏆 **Key Achievements:**
   • Unified detector interface with River integration
   • Advanced metrics: F1@AR, β-score, delay statistics  
   • Comprehensive visualization suite
   • Multi-format result export

📁 **Results**: {export_dir}

🔬 **Next Steps**: Analyze results, compare with literature, extend with additional detectors

Thank you for using Enhanced Concept Drift Benchmark v3! 🚀
""")


In [ ]:

out_csv = "/mnt/data/baseline_v2_results_summary.csv"
df.to_csv(out_csv, index=False)
print("Saved:", out_csv)


OSError: Cannot save file into a non-existent directory: '/mnt/data'

### (Tuỳ chọn) Vẽ timeline drift vs. alarm

In [ ]:

import matplotlib.pyplot as plt

def plot_timeline(r, n_points: int):
    plt.figure(figsize=(10,2))
    for dp in r.drift_points:
        plt.axvline(dp, linestyle="--", alpha=0.6)
    for a in r.alarms:
        plt.axvline(a, color="r", alpha=0.7)
    plt.xlim(0, n_points)
    plt.title(f"{r.stream} / {r.name}: drift (--) vs alarms (red)")
    plt.xlabel("time")
    plt.show()

# Ví dụ
plot_timeline(all_details[("SEA","ADWIN")], N)
plot_timeline(all_details[("LED_abrupt","FHDDMS")], N)
plot_timeline(all_details[("InterchangingRBF","ShapeDD")], N)
